<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Mask_R_CNN_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ftaubner/semantic_features_detection.git

Cloning into 'semantic_features_detection'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 1446 (delta 32), reused 32 (delta 15), pack-reused 1392
Receiving objects: 100% (1446/1446), 114.23 MiB | 30.88 MiB/s, done.
Resolving deltas: 100% (884/884), done.


In [0]:
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21100433&authkey=APrMUGQyaB4np4Q" -O kitti.zip

--2020-04-18 09:46:36--  https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21100433&authkey=APrMUGQyaB4np4Q
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://xwp44a.am.files.1drv.com/y4mCzU_iM5FnOYo5dcFdeqCSAKyovfcWkkmyssL0TWhsyJ50h_Jw6klRxkAaEsFrp4cDi-GDDru2sp5SqjeScpSC-PrWYQLFl9QreUOF325o4Shbhzur982hZLGCTHkQev0in-NaA-JnnfPceT_iXasV6kJYgswm-cUyjQWON3zmLZafpK_w7d_vEea841qxIRGVNPCFBJe37iz9QhC3FrAIA/kitti_dataset.zip?download&psid=1 [following]
--2020-04-18 09:46:37--  https://xwp44a.am.files.1drv.com/y4mCzU_iM5FnOYo5dcFdeqCSAKyovfcWkkmyssL0TWhsyJ50h_Jw6klRxkAaEsFrp4cDi-GDDru2sp5SqjeScpSC-PrWYQLFl9QreUOF325o4Shbhzur982hZLGCTHkQev0in-NaA-JnnfPceT_iXasV6kJYgswm-cUyjQWON3zmLZafpK_w7d_vEea841qxIRGVNPCFBJe37iz9QhC3FrAIA/kitti_dataset.zip?download&psid=1
Resolving xwp44a.am.files.1drv.com (xwp44a.am.fi

In [0]:
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21100964&authkey=AJF_1E9Muh842-0"  -O mapillary_prelim.h5

In [0]:
!unzip -qq /content/kitti.zip -d /content

In [0]:
!pip install imgaug
!pip install Cython
!pip install pycocotools
!pip install wget

In [0]:
%tensorflow_version 1.x
%matplotlib inline
import os
import sys
os.chdir('/content/semantic_features_detection/Notebook')
ROOT_DIR = os.path.abspath('../')
sys.path.append(os.path.join(ROOT_DIR, 'samples'))
import inference

In [0]:
inf_model = inference.Inference(os.path.join(ROOT_DIR, 'mapillary_prelim.h5'))
features, masks = inf_model.predict('/content/dataset/sequences/04/image_2/000001.png')

In [0]:
print(features)

In [0]:
import numpy as np
def masks2instance_im(masks):
    instance_im = np.zeros(shape=[masks.shape[0], masks.shape[1]])
    for i in range(masks.shape[-1]):
        instance_im[masks[...,i]] = (i + 1)
    return instance_im

In [0]:
import glob
import cv2
IMAGE_DIR_LEFT = '/content/dataset/sequences/04/image_2'
IMAGE_DIR_RIGHT = '/content/dataset/sequences/04/image_3'
SAVE_DIR = '/content/Results/Instances'

features = {}
features['results'] = []
image_paths = glob.iglob(os.path.join(IMAGE_DIR_LEFT, '*.*'))
for image_path in image_paths:
    feature, masks = inf_model.predict(image_path)
    features['results'].append(feature)
    # save instance images
    instance_im =  masks2instance_im(masks)
    image_id=os.path.split(image_path)[1][0:-4]
    cv2.imwrite(os.path.join(SAVE_DIR, 'L' + image_id + '.png'), instance_im)


In [0]:
import json
FEATURE_FILE = '/content/results.json'
with open(FEATURE_FILE, 'w') as outfile:
    json.dump(features, outfile)